In [1]:
import json
def load_json(filename):
    with open(filename, mode='r', encoding='utf-8') as f:
    # with open(filename, mode='r') as f:
        data = json.load(f)
    return data

def save_json(data, filename):
    with open(filename, mode='w', encoding='utf-8') as f:
        json.dump(data, f)


def load_lines(filename):
    with open(filename, mode='r', encoding='utf-8') as f:
        return [e.strip("\n") for e in f.readlines()]


In [4]:
# Charades
def convert_charades(old_path, new_path, aux_path):
    aux_data = load_json(aux_path)
    old_data = load_lines(old_path)

    new_data = []
    for line in old_data:
        line = line.lstrip().rstrip()
        if len(line) == 0:
            continue
        video_info, sent = line.split('##')
        vid, s, e = video_info.split(' ')
        duration = aux_data[vid]['duration']

        s = max(round(float(s), 2), 0)
        e = min(round(float(e), 2), duration)
        record = [vid, duration, [s, e], sent]
        new_data.append(record)
        
    save_json(new_data, new_path)

    print(old_path, "->", new_path)
    print(len(old_data), "->", len(new_data))

In [6]:
aux_path = 'data/dataset/charades/charades.json'

old_path = './data/dataset/charades/charades_sta_train.txt'
new_path = './data/dataset/charades_gt/train.json'
convert_charades(old_path, new_path, aux_path)

old_path = './data/dataset/charades/charades_sta_test.txt'
new_path = './data/dataset/charades_gt/test.json'
convert_charades(old_path, new_path, aux_path)

./data/dataset/charades/charades_sta_train.txt -> ./data/dataset/charades_gt/train.json
12408 -> 12408
./data/dataset/charades/charades_sta_test.txt -> ./data/dataset/charades_gt/test.json
3720 -> 3720


In [8]:
# ActivityNet
def convert_anet(old_path, new_path, vlen_path):
    old_data = load_json(old_path)
    vlen = load_json(vlen_path)
    vids = set(vlen.keys())
    
    new_data = []
    for vid in old_data:
        if vid not in vids:
            continue

        sample = old_data[vid]
        timestamps = sample["timestamps"]
        sentences = sample["sentences"]
        duration = sample["duration"]
        for se_frame, sent in zip(timestamps, sentences):
            s, e = se_frame
            s = max(round(s, 2), 0)
            e = min(round(e, 2), duration)
            if s > e:
                print("SKIP:", s, e)
                continue
            record = [vid, duration, [s, e], sent]
            new_data.append(record)
    save_json(new_data, new_path)
    print(old_path, "->", new_path)
    print(len(old_data), "->", len(new_data))


In [10]:
vlen_path = "/storage_fast/rjliang/activitynet/i3d/feature_shapes.json"

old_path = './data/activitynet/train.json'
new_path = './data/anet_i3d_gt/train.json'
convert_anet(old_path, new_path, vlen_path)

old_path = './data/activitynet/val_1.json'
new_path = './data/anet_i3d_gt/test.json'
convert_anet(old_path, new_path, vlen_path)

SKIP: 61.29 60.71
SKIP: 173.44 150.31
./data/activitynet/train.json -> ./data/anet_i3d_gt/train.json
10009 -> 33719
./data/activitynet/val_1.json -> ./data/anet_i3d_gt/test.json
4917 -> 15753


In [4]:
# Tacos
def convert_tacos(old_path, new_path):
    old_data = load_json(old_path)
    vids = list(old_data.keys())
    # vids.sort()

    new_data = []
    for vid in vids:
        sample = old_data[vid]
        timestamps = sample["timestamps"]
        sentences = sample["sentences"]
        fps = sample["fps"]
        num_frames = sample["num_frames"]
        duration = round(num_frames / fps, 2)
        for se_frame, sent in zip(timestamps, sentences):
            s, e = se_frame
            s = max(round(s / num_frames * duration, 2), 0)
            e = min(round(e / num_frames * duration, 2), duration)

            record = [vid[:-4], duration, [s, e], sent]
            new_data.append(record)

    save_json(new_data, new_path)
    print(old_path, "->", new_path)
    print(len(old_data), "->", len(new_data))

In [5]:
old_path = './data/dataset/tacos/train.json'
new_path = './data/dataset/tacos_gt/train.json'

convert_tacos(old_path, new_path)

# old_path = './data/dataset/tacos/test.json'
# new_path = './data/dataset/tacos_gt/test.json'

# convert_tacos(old_path, new_path)

./data/dataset/tacos/train.json -> ./data/dataset/tacos_gt/train.json
75 -> 9790
